<a href="https://colab.research.google.com/github/DiNOV-Tokyo/yolov5/blob/main/DL_torch_GPU_Colab_LocalRunitime_Train_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch実践入門


GPUを使って学習できるかを検証するためのnotebook

## 1.2 モジュール読み込み

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
import time
from collections import OrderedDict

import numpy as np
from tqdm import tqdm
from PIL import Image
import os
import copy

#from torch.utils.tensorboard import SummaryWriter

print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

# GPU/CPUが使えるかどうか確認
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

PyTorch Version:  1.9.0+cu111
Torchvision Version:  0.10.0+cu111
Setup complete. Using torch 1.9.0+cu111 _CudaDeviceProperties(name='NVIDIA GeForce RTX 3070', major=8, minor=6, total_memory=8191MB, multi_processor_count=46)
cuda:0


# 7. 画像分類モデルの構築

### 画像データセット

In [ ]:
from torchvision import datasets
data_path = "./dev/CIFAR10"
cifar10 = datasets.CIFAR10(data_path, train=True, download=False)
cifar10_val = datasets.CIFAR10(data_path, train=False, download=False)

データの正規化のため、mean, stdを求める。

In [ ]:
tensor_cifar10 = datasets.CIFAR10(data_path, train=True, download=False, transform = transforms.ToTensor())
tensor_cifar10_val = datasets.CIFAR10(data_path, train=False, download=False, transform = transforms.ToTensor())

imgs_train = torch.stack([img_t for img_t, _ in tensor_cifar10], dim=3)
#imgs_train.shape

imgs_val = torch.stack([img_v for img_v, _ in tensor_cifar10_val], dim=3)
#imgs_val.shape

mean_train = imgs_train.view(3, -1).mean(dim=1)
std_train = imgs_train.view(3, -1).std(dim=1)

mean_val = imgs_val.view(3, -1).mean(dim=1)
std_val = imgs_val.view(3, -1).std(dim=1)



(tensor([0.4914, 0.4822, 0.4465]),
 tensor([0.2470, 0.2435, 0.2616]),
 tensor([0.4942, 0.4851, 0.4504]),
 tensor([0.2467, 0.2429, 0.2616]))

Datasetの変換

算出したmean, stdで再度データを読み込み

In [ ]:
from torchvision import transforms

transformed_cifar10 = datasets.CIFAR10(data_path, train=True, download=False,
                                       transform=transforms.Compose([
                                                                     transforms.ToTensor(),
                                                                     transforms.Normalize(mean_train, std_train)
                                       ]))
transformed_cifar10_val= datasets.CIFAR10(data_path, train=False, download=False,
                                       transform=transforms.Compose([
                                                                     transforms.ToTensor(),
                                                                     transforms.Normalize(mean_val, std_val)
                                       ]))

#transformed_cifar10 = datasets.CIFAR10(data_path, train=True, download=False,
 #                                      transform=transforms.Compose([
 #                                                                    transforms.ToTensor(),
 #                                                                    transforms.Normalize((0.4915, 0.4823, 0.4468),
 #                                                                                         (0.2470, 0.2435, 0.2616))
 #                                      ]))
#transformed_cifar10_val= datasets.CIFAR10(data_path, train=False, download=False,
 #                                      transform=transforms.Compose([
  #                                                                   transforms.ToTensor(),
  #                                                                   transforms.Normalize((0.4915, 0.4823, 0.4468),
  #                                                                                        (0.2470, 0.2435, 0.2616))
  #                                     ]))


全結合モデル

In [ ]:
import torch.nn as nn

n_out= 2

model = nn.Sequential(
    nn.Linear(
        3072,  #入力32 x 32 x 3
        512,  # 隠れ層のサイズ
    ),
    nn.Tanh(),
    nn.Linear(
        512,
        n_out,
    ),
    nn.Softmax(dim=1)
)

In [ ]:
label_map = {0: 0, 2: 1}
class_names = ['airplane', 'bird']
cifar2 = [(img, label_map[label]) for img, label in transformed_cifar10 if label in [0, 2]]
cifar2_val = [(img, label_map[label]) for img, label in transformed_cifar10_val if label in [0, 2]]

## 8. 畳み込み

## 8.3 nn.Moduleを継承したモデル

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.act1 = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.act2 = nn.Tanh()
        self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(8*8*8, 32)
        self.act3 = nn.Tanh()
        self.fc2 = nn.Linear(32, 2)

    def forward(self, x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = out.view(-1, 8*8*8)
        out = self.act3(self.fc1(out))
        out = self.fc2(out)
        return out


In [ ]:
model = Net()
numel_list = [p.numel() for p in model.parameters()]
sum(numel_list), numel_list

(18090, [432, 16, 1152, 8, 16384, 32, 64, 2])

### 8.3.3 functional API

In [ ]:
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(8*8*8, 32)
        self.fc2 = nn.Linear(32, 2)

    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1, 8*8*8)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out


## 8.3 畳み込みニューラルネットワークの学習

### 8.4.3 GPUGPU上での訓練

In [ ]:
# GPU/CPUが使えるかどうか確認
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device('cpu');
print(device)

cuda:0


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import datetime

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
  
    for epoch in range(1, n_epochs + 1 ):
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device = device) #データをGPUGPUに移動
            labels = labels.to(device = device) #データをGPUGPUに移動
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()
        
        if epoch == 1 or epoch % 10 == 0:

            print("{} Epoch {}: Training loss {}".format(datetime.datetime.now(), epoch, loss_train / len(train_loader)))
          

訓練実行

In [ ]:


train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64, shuffle=True)

model = Net().to(device=device)

learning_rate = 1e-2

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)



/opt/conda/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


2021-09-13 07:50:52.022795 Epoch 1: Training loss 0.5921740949533547
2021-09-13 07:50:54.960562 Epoch 10: Training loss 0.3303600465226325


KeyboardInterrupt: ignored

In [ ]:
# モデル
import torch
import torch.nn as nn
import torch.optim as optim


label_map = {0: 0, 1: 1, 2: 2, 3: 3, 4: 4}
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'bird']
cifar5 = [(img, label_map[label]) for img, label in transformed_cifar10 if label in [0, 1, 2, 3, 4]]
cifar5_val = [(img, label_map[label]) for img, label in transformed_cifar10_val if label in [0, 1, 2, 3, 4]]


train_loader = torch.utils.data.DataLoader(cifar5, batch_size=64, shuffle=True)

n_out= 5

model = nn.Sequential(
    nn.Linear( 3072, 512 ),
    nn.Tanh(),
    nn.Linear( 512, n_out),
    nn.LogSoftmax(dim=1)
)

import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(8*8*8, 32)
        self.fc2 = nn.Linear(32, n_out)

    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1, 8*8*8)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out




import datetime

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
  
    for epoch in range(1, n_epochs + 1 ):
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device = device) #データをGPUGPUに移動
            labels = labels.to(device = device) #データをGPUGPUに移動
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()
        
        if epoch == 1 or epoch % 10 == 0:

            print("{} Epoch {}: Training loss {}".format(datetime.datetime.now(), epoch, loss_train / len(train_loader)))


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device('cpu')

train_loader = torch.utils.data.DataLoader(cifar5, batch_size=64, shuffle=True)

model = Net().to(device=device)

learning_rate = 1e-2

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

loss_fn = nn.CrossEntropyLoss()

training_loop(
#    n_epochs = 30,
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)


Exception ignored in: <function tqdm.__del__ at 0x7f3d09558d30>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/tqdm/std.py", line 1152, in __del__
    self.close()
  File "/opt/conda/lib/python3.9/site-packages/tqdm/notebook.py", line 286, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm' object has no attribute 'disp'


RuntimeError: ignored